In [1]:
'''
input: file with empty/nonempty road and latency for each frame
output: file(at folder of "fps_converted") with accumulative "empty road %" and fps at the time of each frame

'''
import pandas as pd
import os
import numpy as np
import re

################## choose files to convert here ##############
# 1. cuda +cuda (stage1 +stage2):
# file = 'stage1_erd_cuda+stage2_yolov5_cuda.csv'
# file ='stage1_erd_cuda+stage2_efficientDet_cuda.csv'
# file = 'stage1_erd_cuda+stage2_ssd_cuda.csv'
#
# 2. cpu + cuda (stage1 +stage2):
# file = 'stage1_erd_cpu+stage2_yolov5_cuda.csv'
# file = 'stage1_erd_cpu+stage2_ssd_cuda.csv'
file ='stage1_erd_cpu+stage2_efficientDet_cuda.csv'

# 3. cpu + cpu (stage1 +stage2):
# file = 'stage1_erd_cpu+stage2_efficientDet_cpu.csv'
# file = 'stage1_erd_cpu+stage2_ssd_cpu.csv'
# file = 'stage1_erd_cpu+stage2_yolov5_cpu.csv'
##############################################################



file_prefix = re.findall(r'(.+).csv',file)[0]
data = pd.read_csv(os.path.join('../result/raw_data', file), index_col = None)
data = data[100:]  ## take out the gpu warm up 
data['empty_road_detection_result']= data['empty_road_detection_result'].astype(int) ### value 0 means road is empty
data['latency_whole']=data['latency_whole'].astype(float)
er_sum = 0
er_list = data['empty_road_detection_result'].tolist()
latency_list = data['latency_whole'].tolist()
# accum_ave_latency = []
fps_list = []
empty_rate_list, accum_ave_latency, fps_list  = [], [], []
for cnt in range(1,len(er_list)):
    er_sum += er_list[cnt] ## 0 means road is empty
    empty_rate_list.append(1-er_sum/cnt)  
    ave_latency = np.mean(latency_list[:cnt])
    accum_ave_latency.append(ave_latency)
    fps_list.append(1/ave_latency)
# accum_ave_latency=accum_ave_latency[1:]

result = pd.DataFrame(columns = ['road_empty %', 'average_latency','fps'])
result['road_empty %'] = empty_rate_list
result['average_latency'] = accum_ave_latency
result['fps'] = fps_list

result.to_csv('temp.csv')
result.to_csv('../result/fps_converted/'+'fps_result_'+file_prefix+'.csv')